### Table 생성

In [1]:
import boto3
def create_indices_table(dynamodb=None):
	if not dynamodb:
		dynamodb = boto3.resource('dynamodb',
                          aws_access_key_id="anything",
                          aws_secret_access_key="anything",
                          region_name="us-west-2",
                          endpoint_url="http://localhost:8000")
		table_name = 'StockPrice'
		table = dynamodb.Table(table_name)
		try:
			table.delete()
		except dynamodb.meta.client.exceptions.ResourceNotFoundException:
			pass

		table = dynamodb.create_table(TableName=table_name,
			KeySchema=[
			{'AttributeName': 'StockName','KeyType': 'HASH' # Partition key
			},
			{'AttributeName': 'Date','KeyType': 'RANGE' # Sort key
			}
			],
			AttributeDefinitions=[
			{'AttributeName': 'StockName','AttributeType': 'S'},
			{'AttributeName': 'Date','AttributeType': 'S'},
			],
			ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10}
		)
	return table
if __name__ == '__main__':
	indices_table = create_indices_table()
	print("Table status:", indices_table.table_status)

ModuleNotFoundError: ignored

### Table item 추가, 쿼리, 수정, 삭제하기

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from decimal import Decimal

def write_data(stockname, date, value, dynamodb):
    table = dynamodb.Table('StockPrice')
    item = {'StockName':stockname,
            'Date':date,
            'Value':Decimal(value)}
    table.put_item(Item=item)
    print('item', stockname, 'added!')
    print('-------')

def query_data_by_name(stockname, dynamodb):
    table = dynamodb.Table('StockPrice')
    response = table.query(
        KeyConditionExpression =
            Key('StockName').eq(stockname))
    print('Querying items for', 'StockName =', stockname)
    print(response['Items'])
    print('-------')

def query_data_by_date(stockname, date_from, date_to, dynamodb):
    table = dynamodb.Table('StockPrice')
    response = table.query(
        KeyConditionExpression =
            Key('StockName').eq(stockname)
            & Key('Date').between(date_from, date_to))
    print('Querying items for', 'StockName =',
          stockname, date_from, '~', date_to)
    print(response['Items'])
    print('-------')

def update_data(stockname, date, new_value, dynamodb):
    table = dynamodb.Table('StockPrice')
    response = table.update_item(
        Key={'StockName':stockname,'Date':date},
        UpdateExpression='Set #V=:v',
        ExpressionAttributeNames={
            '#V':'Value'},
        ExpressionAttributeValues={
            ':v':Decimal(new_value)},
        ReturnValues='UPDATED_NEW'
    )
    print('item', stockname, 'updated!')
    print('-------')

def remove_data(stockname, date, dynamodb):
    table = dynamodb.Table('StockPrice')
    response = table.delete_item(
        Key={'StockName':stockname, 'Date':date}
    )
    print('item', stockname, 'deleted!')
    print('-------')

date = '2023-04-20'
stockname = 'Samsung Electronics'
value = 65300
dynamodb = boto3.resource('dynamodb',
                          aws_access_key_id="anything",
                          aws_secret_access_key="anything",
                          region_name="ap-northeast-2",
                          endpoint_url="http://localhost:8000")

write_data(stockname, date, value, dynamodb)
query_data_by_name(stockname, dynamodb)
query_data_by_date(stockname, '2023-04-19', '2023-04-21', dynamodb)
query_data_by_date(stockname, '2023-04-21', '2023-04-23', dynamodb)
update_data(stockname, date, 65400, dynamodb)
query_data_by_name(stockname, dynamodb)
remove_data(stockname, date, dynamodb)
query_data_by_name(stockname, dynamodb)

item Samsung Electronics added!
-------
Querying items for StockName = Samsung Electronics
[{'Date': '2023-04-20', 'StockName': 'Samsung Electronics', 'Value': Decimal('65300')}]
-------
Querying items for StockName = Samsung Electronics 2023-04-19 ~ 2023-04-21
[{'Date': '2023-04-20', 'StockName': 'Samsung Electronics', 'Value': Decimal('65300')}]
-------
Querying items for StockName = Samsung Electronics 2023-04-21 ~ 2023-04-23
[]
-------
item Samsung Electronics updated!
-------
Querying items for StockName = Samsung Electronics
[{'Date': '2023-04-20', 'StockName': 'Samsung Electronics', 'Value': Decimal('65400')}]
-------
item Samsung Electronics deleted!
-------
Querying items for StockName = Samsung Electronics
[]
-------
item Samsung Electronics added!
-------
